In [1]:
corpus = [
    "रोज़ सुबह, मैं उठकर योग करता हूँ।",
    "बहुत अच्छा गायक है वह।",
    "आपका स्वागत है, हम सभी आपका स्वागत करते हैं।",
    "उसने दोस्त को हंसी का कारण बहुत अच्छे से बताया।",
    "हम सभी मिलकर बहुत मजेदार पार्टी मनाएंगे।"
]

In [2]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2")

/Users/workbook01/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/workbook01/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from collections import defaultdict

word_freqs = defaultdict(int)

for text in corpus:
    words_with_offsets = tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str(text)
    new_words = [word for word, offset in words_with_offsets]
    for word in new_words:
        word_freqs[word] += 1

print(word_freqs)

In [7]:
alphabet = []

for word in word_freqs.keys():
    for letter in word:
        if letter not in alphabet:
            alphabet.append(letter)
alphabet.sort()

print(alphabet)

[',', '£', '¤', '¥', '¦', '¨', 'ª', '¬', '®', '¯', '°', '²', 'µ', '¸', '¹', '¼', '¾', '¿', 'à', 'Ġ', 'Ģ', 'ģ', 'Ĥ', 'ħ', 'Ĩ', 'ĩ', 'Ī', 'ī', 'ĭ', 'į', 'ı', 'ķ', 'Ĺ', 'ļ', 'Ľ', 'ľ', 'Ł', 'ł', 'Ń']


In [8]:
vocab = ["<|endoftext|>"] + alphabet.copy()
splits = {word: [c for c in word] for word in word_freqs.keys()}

In [9]:
def compute_pair_freqs(splits):
    pair_freqs = defaultdict(int)
    for word, freq in word_freqs.items():
        split = splits[word]
        if len(split) == 1:
            continue
        for i in range(len(split) - 1):
            pair = (split[i], split[i + 1])
            pair_freqs[pair] += freq
    return pair_freqs

In [10]:
pair_freqs = compute_pair_freqs(splits)

for i, key in enumerate(pair_freqs.keys()):
    print(f"{key}: {pair_freqs[key]}")
    if i >= 5:
        break

('à', '¤'): 117
('¤', '°'): 8
('à', '¥'): 34
('¥', 'ĭ'): 4
('¤', 'ľ'): 2
('¤', '¼'): 1


In [11]:
best_pair = ""
max_freq = None

for pair, freq in pair_freqs.items():
    if max_freq is None or max_freq < freq:
        best_pair = pair
        max_freq = freq

print(best_pair, max_freq)

('à', '¤') 117


In [12]:
merges = {("à", "¤"): "à¤"}
vocab.append("à¤")